<a href="https://colab.research.google.com/github/simplifine-llm/Simplifine/blob/main/examples/sft_quick_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a quick guide on getting started with Simplifine!

Below is an example of sending a supervised fine-tuning job to Simplifine's hosted servers.

First, we start by downloading Simplifine's latest version from github.

In [1]:
!pip install git+https://github.com/simplifine-llm/Simplifine.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 314.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

Supervised fine tuning is a useful method to fine-tune a model for generating formatted answers, based on the provided input.

An example would be to generate an answer based on provided context.

An example would be:

QUESTION: What is the capital France?

CONTEXT: France has had its capital as Paris for some time now!

ANSWER: Paris is the capital of France.

In this example, you would want the model to fill in for the answer, having provided it with the question and context.

In this example, the Alpaca dataset will be used. We will use the following prompt template:

```
'''### TITLE: {title}\n ### ABSTRACT: {abstract}\n ###EXPLANATION: {explanation}'''
```

Then as mentioned, we want the model to fill in the text for answer, so we asign this to a response template:



```
response_template='\n ###EXPLANATION:'
```

In the example below, we use our own dataset. This dataset should be a python dictionary, which should include the keys that are required to populate the template you provided. You can also use any dataset hosted on huggingface (some require authentication/tokens)

Next, we try and fine-tune a GPT-2 model, here on colab using a GPU.

In [2]:
from simplifine_alpha import train_engine
import wandb
import os

# disabling WandB logging, change if you'd like to have one.
# Note that you will need a wandb token.
wandb.init(mode='disabled')

# You can provided a HF dataset name.
# be sure to change the keys, response template and tempalte accordingly.
template = '''### TITLE: {title}\n ### ABSTRACT: {abstract}\n ###EXPLANATION: {explanation}'''
response_template='\n ###EXPLANATION:'
keys = ['title', 'abstract', 'explanation']
dataset_name=''

# you can change the model. bigger models might throw OOM errors.
model_name = 'EleutherAI/pythia-160m'

if True:  # change this if you want to try this on a dataset on huggingface!
  from_hf = False
  data = {
      'title':['title 1', 'title 2', 'title 3']*10,
      'abstract':['abstract 1', 'abstract 2', 'abstract 3']*10,
      'explanation':['explanation 1', 'explanation 2', 'explanation 3']*10
  }

train_engine.hf_sft(model_name, from_hf=from_hf, dataset_name=dataset_name,
        keys = keys, data = data,
        template = template,
        response_template=response_template, zero=False, ddp=False, gradient_accumulation_steps=4, fp16=True, max_seq_length=2048)

[2024-07-26 14:56:43,463] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

Using CUDA


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:505: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(


Step,Training Loss


In the example above, a small Pythia model (160m parameters) on a L4 GPU. Note that we do not use any adapters e.g. LoRA.
In the next step, we show how simplifine allows to carry out the same thing, but on GPU clusters. This will use functions of train_utils.

By using this command, you can manually pass the parallelization method.

If you have a model that is small enough, try using DDP. In this method, each processor (fansy word for GPU!) has a replica of the model and attends to a different sample.

You can also utilize ZeRO from DeepSpeed. With this, you can shard the model parameters, activation states and gradients across the GPUs. You also have the option to offload some to CPUs, at the expense of lower throughput.

### Using DDP to train
The example below uses DDP to distribute the training process.

In [3]:
from simplifine_alpha.train_utils import sft_train_cloud

# you would need a simplifine API key. contact us for one for free! :)
api_key = 'SimpTest'

# simply pass all the arguements you used above, and change ddp ot zero if you want parallelization.
sft_train_cloud(model_name = model_name, from_hf=from_hf, dataset_name=dataset_name,
        keys = keys, data = data,
        template = template, job_name='test', api_key=api_key,
        response_template=response_template, use_zero=False, use_ddp=True)

After sending the query, you can check the status of your jobs. Note that the status is one of the three options:
```text
status = complete|in progress|pending
```

In [6]:
from simplifine_alpha.train_utils import get_job_status

status = get_job_status(api_key)

for num,i in enumerate(status):
  print(f'Number {num} status: {i}\n')

Number 0 status: {'job_id': '52c7c957-e712-46a3-a5b9-f479f9065d2d', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 1 status: {'job_id': '85bff99b-f469-4041-a955-9bd6eb6a5ec5', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 2 status: {'job_id': '2fe4a5a4-a2ad-41c2-b2e9-ce9f15d3ccf1', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 3 status: {'job_id': '85ce697f-e8ce-45c0-8c92-8decfee2f9bd', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 4 status: {'job_id': '93cac1ce-6cba-4819-b70d-4cf94e99c67a', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 5 status: {'job_id': '5f05254a-5d2f-46c9-9843-da1b215e91b4', 'job_name': 'test', 'status': 'completed'}



###Logs
Getting the logs of a certain job requires the job_id. copy the relevant job_id and use it to retireve the logs. This will be usefull for seeing if there were any errors.

In [8]:
from simplifine_alpha.train_utils import get_train_logs

# getting the job_id of the last job
job_id = status[-1]['job_id']

logs = get_train_logs(api_key, job_id)
print(logs)

{'response': "W0726 14:57:06.123000 125084996468864 torch/distributed/run.py:757] \nW0726 14:57:06.123000 125084996468864 torch/distributed/run.py:757] *****************************************\nW0726 14:57:06.123000 125084996468864 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. \nW0726 14:57:06.123000 125084996468864 torch/distributed/run.py:757] *****************************************\n[2024-07-26 14:57:09,432] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)\n[2024-07-26 14:57:09,736] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)\n[2024-07-26 14:57:09,915] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)\n[2024-07-26 14:57:10,063] [INFO] [real_accelerator.p

### Using ZeRO
ZeRO is a strong tool when a model cannot fit on GPU memory, so it is sharded across them (parameters, gradients and activations). Further memory reduction could be by enabling fp16/bf16, and gradient_checkpointing.

In [10]:
# This time, we just change the use_zero arg to True, and opposite to use_ddp.
sft_train_cloud(model_name = model_name, from_hf=from_hf, dataset_name=dataset_name,
        keys = keys, data = data,
        template = template, job_name='zero_example', api_key=api_key,
        response_template=response_template, use_zero=True, use_ddp=False)

In [11]:
# repeat the same step of extracting jobs and ids
status = get_job_status(api_key)

for num,i in enumerate(status):
  print(f'Number {num} status: {i}\n')

Number 0 status: {'job_id': '52c7c957-e712-46a3-a5b9-f479f9065d2d', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 1 status: {'job_id': '85bff99b-f469-4041-a955-9bd6eb6a5ec5', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 2 status: {'job_id': '2fe4a5a4-a2ad-41c2-b2e9-ce9f15d3ccf1', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 3 status: {'job_id': '85ce697f-e8ce-45c0-8c92-8decfee2f9bd', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 4 status: {'job_id': '93cac1ce-6cba-4819-b70d-4cf94e99c67a', 'job_name': 'sft_cloud_example', 'status': 'completed'}

Number 5 status: {'job_id': '5f05254a-5d2f-46c9-9843-da1b215e91b4', 'job_name': 'test', 'status': 'completed'}

Number 6 status: {'job_id': 'd46aecbb-84a0-40fc-9913-43065b3cb4f2', 'job_name': 'test', 'status': 'completed'}

Number 7 status: {'job_id': '46a0bde6-e526-49cf-a004-330ddd7ed830', 'job_name': 'zero_example', 'status': 'in progress'}



In [12]:
# extracting logs again
job_id = status[-1]['job_id']

logs = get_train_logs(api_key, job_id)
print(logs)

{'response': "W0726 15:05:32.888000 123372810014848 torch/distributed/run.py:757] \nW0726 15:05:32.888000 123372810014848 torch/distributed/run.py:757] *****************************************\nW0726 15:05:32.888000 123372810014848 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. \nW0726 15:05:32.888000 123372810014848 torch/distributed/run.py:757] *****************************************\n[2024-07-26 15:05:36,275] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)\n[2024-07-26 15:05:36,669] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)\n[2024-07-26 15:05:36,710] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)\n[2024-07-26 15:05:36,874] [INFO] [real_accelerator.p